In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader, Dataset
from torch.autograd import Variable 
import torchvision
from torchvision import transforms
from MyDataLoader import Dataset
from torch import utils

In [2]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
torch.backends.cudnn.benchmark = True
use_cuda

True

In [3]:
class LogisticRegressionModel(torch.nn.Module): 
  
    def __init__(self, input_dim): 
        super(LogisticRegressionModel, self).__init__()
        self.linear = torch.nn.Linear(input_dim, 1)
    def forward(self, x): 
        y = F.sigmoid(self.linear(x))
        return y

In [4]:
train_data_file = "Brunel_BdJPsiKs_MagU_30k.csv"
data = pd.read_csv(train_data_file)
features = data.drop(["is_true_seed","has_MCParticle",
                      "is_downstream_reconstructible",
                      "has_MCParticle_not_electron",
                     "is_downstream_reconstructible_not_electron",
                     "seed_mva_value", "seed_p",
                     "seed_nbIT", "seed_tx", "seed_ty"] , axis=1).astype(np.float64)
targets = torch.FloatTensor(data.is_downstream_reconstructible.values)
features = torch.FloatTensor(features.drop(features.columns[0], axis=1).values)

In [5]:
batch_size = 200
epochs = 5000
n_iters = epochs * (len(targets) / batch_size)
input_dim = 6
output_dim = 1
lr_rate = 0.001

In [6]:
model = LogisticRegressionModel(input_dim).cuda()
criterion = torch.nn.BCELoss(reduction='mean')
optimizer = torch.optim.SGD(model.parameters(), lr=lr_rate)

In [7]:
def transform(x):
    x * torch.FloatTensor([1/9,1/7000,1/30,1/4,1/2000,1/2000]) + torch.FloatTensor([0,0,0,-8,0,0])

In [8]:
training_set = Dataset(features, targets, transform)
training_generator = utils.data.DataLoader(training_set, batch_size = batch_size, shuffle = True)

In [10]:
for epoch in range(epochs):
    for features, targets in training_generator:
        features, targets = features.to(device), targets.to(device)
        optimizer.zero_grad()
        pred = model(features)
        loss = criterion(pred, targets)
        loss.backward()
        optimizer.step()
    print('epoch {}, loss {}'.format(epoch, loss.data))
    

TypeError: batch must contain tensors, numbers, dicts or lists; found <class 'NoneType'>